In [2]:
# Импорт библиотек
import numpy as np
from datasets import Dataset
import nltk
from transformers import T5TokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
from transformers.optimization import Adafactor, AdafactorSchedule
from datasets import load_dataset, load_metric
import pandas as pd
from tqdm import tqdm

In [2]:
# загрузка параметров
raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")
nltk.download('punkt')

Found cached dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


  0%|          | 0/3 [00:00<?, ?it/s]

/tmp/ipykernel_23680/1079567798.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Ввести свой ключ huggingface hyb
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Определение параметров
REPO = "speller-t5-90"  # Введите наазвание название репозитория
MODEL_NAME =  "sberbank-ai/ruT5-base" # Введите наазвание выбранной модели из хаба
MAX_INPUT = 256  # Введите максимальную длинну входных данных  в токенах (длинна входных фраз в словах (можно считать полслова токен))
MAX_OUTPUT  = 256 # Введите максимальную длинну прогнозов в токенах (можно уменьшить для задач суммризации или других задач где выход короче)
BATCH_SIZE = 8

In [45]:
REPO

'speller-t5-90'

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
%history
import pandas as pd
# data1 = pd.read_csv("./dataset_big.csv", index_col=0)[:15000]
data = pd.read_csv("./dataset_small 2.csv", index_col=0).drop_duplicates()
test_data = pd.read_csv("./test_dataset.csv", index_col=0).drop_duplicates()
data.head(2)

# Импорт библиотек
import numpy as np
from datasets import Dataset
import nltk
from transformers import T5TokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
from transformers.optimization import Adafactor, AdafactorSchedule
from datasets import load_dataset, load_metric
import pandas as pd
from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import pandas as pd
# data = pd.read_csv("./dataset_big.csv", index_col=0)[:15000]
data = pd.read_csv("./dataset_small 2.csv", index_col=0).drop_duplicates()
test_data = pd.read_csv("./test_dataset.csv", index_col=0).drop_duplicates()
data.head(2)
from transformers import T5TokenizerFast, AutoModelForSeq2SeqLM

MODEL_NAME_ = 'summervent/speller-t5-90'
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME_)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_).to(device);
test_data
test_data = pd.read_csv("./dataset_small 2.csv", index_col

,source,target
0,очень классная тетка ктобы что не говорил.,очень классная тетка кто бы что ни говорил
1,Может выгоднее втулку продать и купить колесо ...,Может выгоднее втулку продать и купить колесо ...


In [147]:
print(data.shape)
print(test_data.shape)

(117417, 2)
(2005, 2)


In [5]:
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from transformers import AutoTokenizer, AutoModel
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
# model.resize_token_embeddings(len(tokenizer));
# model = nn.DataParallel(model).to(device);

In [27]:
input_data = list(data.source)
output_data = list(data.target)
task_prefix = "Исправь: "
input_sequences = input_data
encoding = tokenizer( [task_prefix + sequence for sequence in input_sequences], padding="longest", \
                     max_length=MAX_INPUT, truncation=True, return_tensors="pt", )
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask
target_encoding = tokenizer(output_data, padding='max_length', max_length=MAX_OUTPUT, truncation=True)
labels = target_encoding.input_ids
labels = torch.tensor(labels)
labels[labels == tokenizer.pad_token_id] = -100
data = Dataset.from_pandas(pd.DataFrame({'input_ids': list(np.array(input_ids)), 'attention_mask': list(np.array(attention_mask)), \
                                         'labels': list(np.array(labels))}))
data = data.train_test_split(0.001)

In [165]:
data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 117299
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 118
    })
})

In [166]:
max(list(map(lambda x:len(x.split()),input_data)))

101

In [167]:
data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 117299
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 118
    })
})

In [38]:
model.config.max_length = MAX_OUTPUT  # по умолчанию 20, поэтому во всех моделях прогнозы обрезаются выходные последовательности
# Закоментить после первого соъранения в репозиторий свой необъязательно
tokenizer.push_to_hub(REPO)

CommitInfo(commit_url='https://huggingface.co/summervent/speller-t5-90/commit/edbd9ffd221751f6183355436f20fe1e9bf855d6', commit_message='Upload tokenizer', commit_description='', oid='edbd9ffd221751f6183355436f20fe1e9bf855d6', pr_url=None, pr_revision=None, pr_num=None)

In [29]:
train = data['train']
test = data['test'] # Уменьшил так тестовыу. выборку чтоб не ждать долго расчет ошибок между эпохами

In [30]:
test

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 118
})

In [31]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) #return_tensors="tf"

In [32]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  # Replace -100 in the labels as we can't decode them.
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  
  # Rouge expects a newline after each sentence
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
  
  result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  # Extract a few results
  result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
  
  # Add mean generated length
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result["gen_len"] = np.mean(prediction_lens)
  
  return {k: round(v, 4) for k, v in result.items()}

In [33]:
REPO

'speller-t5-90'

In [172]:
training_args = Seq2SeqTrainingArguments(
  output_dir = REPO,
  #overwrite_output_dir=True,
  evaluation_strategy='steps',
  #learning_rate=2e-5,
  logging_steps=10,
  eval_steps=500,
  save_steps=5000,
  num_train_epochs=1,
  predict_with_generate=True,
  per_device_train_batch_size=BATCH_SIZE,
  per_device_eval_batch_size=BATCH_SIZE,
  fp16=True,
  save_total_limit=2,
  #generation_max_length=256,
  #generation_num_beams=4,
  weight_decay=0.005,
  # logging_strategy='steps',
  #logging_dir='logs',
  push_to_hub=True,
  report_to='tensorboard'
)

In [173]:
# Выберем вручную оптимизатор. Т5 в оригинальной архитектуре использует Адафактор оптимизатор
optimizer = Adafactor(
    model.parameters(),
    lr=1e-5,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False,
)
lr_scheduler = AdafactorSchedule(optimizer)

In [174]:
trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset = data['train'],
  eval_dataset = data['test'],
  optimizers = (optimizer, lr_scheduler),
  tokenizer = tokenizer,
  compute_metrics=compute_metrics,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [125]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
# or 
logging.disable(logging.WARNING)
trainer.train()   # тренируем на data1

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,0.350200,0.285244,18.556700,6.185600,18.556700,18.556700,34.072200
1000,0.271900,0.284167,18.556700,6.185600,18.556700,18.556700,33.969100
1500,0.277300,0.286867,18.556700,6.185600,18.556700,18.556700,33.814400
2000,0.305600,0.275859,18.556700,6.185600,18.556700,18.556700,33.804100
2500,0.462500,0.253244,18.556700,6.185600,18.556700,18.556700,33.886600
3000,0.381200,0.257706,18.556700,6.185600,18.556700,18.556700,33.866000
3500,0.341900,0.249512,18.556700,6.185600,18.556700,18.556700,34.268000
4000,0.352000,0.251150,18.556700,6.185600,18.556700,18.556700,33.886600
4500,0.288700,0.245136,18.556700,6.185600,18.556700,18.556700,33.835100
5000,0.347300,0.248377,18.556700,6.185600,18.556700,18.556700,33.804100


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=5995, training_loss=0.32485827737494843, metrics={'train_runtime': 1950.6203, 'train_samples_per_second': 24.582, 'train_steps_per_second': 3.073, 'total_flos': 1.3002916082688e+16, 'train_loss': 0.32485827737494843, 'epoch': 5.0})

In [175]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
# or 
logging.disable(logging.WARNING)
trainer.train()   #продолжаем трейн на data

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,0.343500,0.209993,19.350300,8.389800,19.420900,19.491500,41.449200
1000,0.324500,0.210189,19.597500,8.757100,19.703400,19.774000,41.194900
1500,0.377700,0.200966,19.350300,8.389800,19.420900,19.491500,41.000000
2000,0.364300,0.197958,19.350300,8.389800,19.420900,19.491500,41.059300
2500,0.321200,0.198607,19.209000,8.206200,19.279700,19.279700,41.152500
3000,0.418100,0.189598,19.350300,8.389800,19.420900,19.491500,42.237300
3500,0.317500,0.187852,19.350300,8.389800,19.420900,19.491500,41.457600
4000,0.339900,0.183755,19.350300,8.389800,19.420900,19.491500,41.110200
4500,0.314000,0.183712,19.350300,8.389800,19.420900,19.491500,41.033900
5000,0.306300,0.179564,19.350300,8.389800,19.420900,19.491500,40.940700


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=14663, training_loss=0.311196438504267, metrics={'train_runtime': 5220.203, 'train_samples_per_second': 22.47, 'train_steps_per_second': 2.809, 'total_flos': 3.571507580239872e+16, 'train_loss': 0.311196438504267, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub()

In [58]:
MODEL_NAME_

'summervent/speller-t5-90'

In [4]:
from transformers import T5TokenizerFast, AutoModelForSeq2SeqLM

MODEL_NAME_ = 'summervent/speller-t5-90'
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME_)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_).to(device);

In [5]:
test_data

,source,target
0,есть у вас оформленый и подписаный мною заказ,есть у вас оформленный и подписанный мною заказ
1,вот в инете откапал такую интеерсную статейку ...,вот в инете откопал такую интересную статейку ...
2,я на всю жизнь запомню свое первое купание в з...,я на всю жизнь запомню свое первое купание в з...
3,думаем что не ошибемся если скажем что выставк...,думаем что не ошибемся если скажем что выставк...
4,судьба человека может складываться очень разно...,судьба человека может складываться очень разно...
...,...,...
2003,спасибо вам огромное за нормальную новость о е...,спасибо вам огромное за нормальную новость о е...
2004,более захватывающее и наглядное обучение возмо...,более захватывающее и наглядное обучение возмо...
2005,и вобщем-то все понятно на фоне слухов застави...,и в общем-то все понятно на фоне слухов застав...
2006,но всему есть придел,но всему есть предел


In [6]:
test_data = pd.read_csv("./test_dataset.csv", index_col=0).drop_duplicates()

In [7]:
MAX_INPUT = 256
input_sequences = list(test_data.source.values)

task_prefix = "Исправь: "
encoded = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=MAX_INPUT,
    truncation=True,
    return_tensors="pt",
)
test_loader = torch.utils.data.DataLoader(input_sequences, batch_size=8)

predicts__ = []
for input_sequences in tqdm(test_loader):
    encoded = tokenizer(
        [task_prefix + sequence for sequence in input_sequences],
        padding="longest",
        max_length=MAX_INPUT,
        truncation=True,
        return_tensors="pt",
    )

    predicts_ = model.generate(**encoded.to(device), max_length=len(encoded['input_ids'][0]), repetition_penalty=1.0)
    
    predicts_ = tokenizer.batch_decode(predicts_, skip_special_tokens=True)
    predicts__ += predicts_


100%|██████████| 251/251 [03:39<00:00,  1.14it/s]


In [8]:
idx = 18
print(predicts__[idx])

через пару минут он возвращается и приносит два стакана честно говоря я так и не понял почему два ну да ладно нам не привыкать и когда я ему объяснил что нужно было три он долго порывался сбегать еще раз но видя какие стаканы он принес я понял что лучше пить из горла а стаканы или даже точнее бокалы это отдельная история это были огромные бокалы под коктель маргарита


In [9]:
test_data.reset_index().loc[18]['target']

'через пару минут он возвращается и приносит два стакана честно говоря я так и не понял почему два ну да ладно нам не привыкать и когда я ему объяснил что нужно было три он долго порывался сбегать еще раз но видя какие стаканы он принес я понял что лучше пить из горла а стаканы или даже точнее бокалы эти это отдельная история это были огромные бокалы под коктейль маргарита'

In [10]:
from evaluate_spell import extract_words, make_corrections_data, measure_quality


OUTPUT_DIFF = False


source_sents = [
    extract_words(line.strip().strip("\ufeff"))
    for line in test_data.source
    if line.strip().strip("\ufeff") != ""
]
correct_sents = [
    extract_words(line.strip().strip("\ufeff"))
    for line in test_data.target
    if line.strip().strip("\ufeff") != ""
]
answer_sents = [extract_words(line.strip().strip("\ufeff")) for line in predicts__]
etalon_corrections, answer_corrections = make_corrections_data(
    source_sents, correct_sents, answer_sents
)
TP, precision, recall, f_measure = measure_quality(
    etalon_corrections, answer_corrections
)
print(
    f"Precision={100*precision:.2f} Recall={100 * recall:.2f} FMeasure={100 * f_measure:.2f}"
)
print(TP, len(answer_corrections), len(etalon_corrections))
if OUTPUT_DIFF:
    output_differences(
        "diff.txt",
        source_sents,
        correct_sents,
        answer_sents,
        etalon_corrections,
        answer_corrections,
    );

0 
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 
101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 
141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 
161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 
181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 
221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 
241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 
261 262 263 264 265 266 267 268 269 270 271 272 273 

In [11]:
count_correct = 0
for answer, correct in zip(correct_sents, answer_sents):
    if answer == correct:
        count_correct += 1
print(count_correct /len(test_data))

0.6952618453865337
